#Imports

In [ ]:
from google.colab import drive
from IPython.display import Image

import folium
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

#Mount Drive

In [ ]:
drive.mount('/content/drive')

#Constants

In [ ]:
dir = "/content/drive/MyDrive/Thesis/"

In [ ]:
# sources = ['CHIRPS', 'MODIS', 'S1', 'S2']
# sources = ['MODIS']
sources = ['S1']

#Dataframes

In [ ]:
Image(dir + "images/aoi.png")

In [ ]:
points_df = pd.read_csv(dir + 'output/points.csv')
points_df = points_df.drop('Unnamed: 0', 1)
print(points_df)

In [ ]:
points_df = points_df[['lat', 'lon']]
print(points_df)

In [ ]:
map = folium.Map(location=[points_df.lat.mean(), 
                           points_df.lon.mean()], 
                 zoom_start=11, 
                 control_scale=True)

points_df.apply(lambda row: folium.CircleMarker(location=[row["lat"], row["lon"]], 
                                                radius=1, 
                                                color='blue').add_to(map),
                axis=1)

map

In [ ]:
dfs = {}

for s in sources:
  dfs[s] = {}
  path = dir + s
  files = sorted([f for f in os.listdir(path) if f.endswith('all.csv')])
  print(files)
  for f in files:
    index = f.split('__')[0]
    dfs[s][index] = pd.read_csv(path + '/' + f)
    dfs[s][index] = dfs[s][index].drop('Unnamed: 0', 1)
    dfs[s][index]['date'] = pd.to_datetime(dfs[s][index]['date'])
    # dfs[s][index].set_index('date', inplace=True)
    # dfs[s][index].sort_index(inplace=True)

    # Filter to: 2017-05 until 2021-10 (4.5 years)
    dfs[s][index] = dfs[s][index][dfs[s][index]['date'].dt.year.isin([2017]) & dfs[s][index]['date'].dt.month.isin([5, 6, 7, 8, 9, 10, 11, 12]) | 
                                  dfs[s][index]['date'].dt.year.isin([2018, 2019, 2020]) | 
                                  dfs[s][index]['date'].dt.year.isin([2021]) & dfs[s][index]['date'].dt.month.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])]
    dfs[s][index].sort_index(inplace=True)

In [ ]:
print(dfs['CHIRPS']['precipitation'])

In [ ]:
# print(dfs['MODIS']['lst'])

In [ ]:
print(dfs['MODIS']['lst_c'])

In [ ]:
print(dfs['MODIS']['quality'])

In [ ]:
print(dfs['S1']['b_asc_vh'])

In [ ]:
print(dfs['S1']['b_des_vh'])

In [ ]:
print(dfs['S1']['b_des_vv'])

In [ ]:
print(dfs['S2']['cloudfree'])

In [ ]:
print(dfs['S2']['ndvi'])

##CHIRPS (Precipitation)

###Aggregate Weekly

In [ ]:
dfs['CHIRPS']['precipitation_ma'] = dfs['CHIRPS']['precipitation'].copy()

In [ ]:
dfs['CHIRPS'].keys()

In [ ]:
dfs['CHIRPS']['precipitation_ma'] = dfs['CHIRPS']['precipitation_ma'].groupby([pd.Grouper(key='date', freq='W-MON')]).sum().reset_index().sort_values('date')
dfs['CHIRPS']['precipitation_ma'].sort_index(inplace=True)

In [ ]:
print(dfs['CHIRPS']['precipitation_ma'])

In [ ]:
print(dfs['CHIRPS']['precipitation'][:21])

In [ ]:
fig = go.Figure()

fig = px.bar(dfs['CHIRPS']['precipitation'], x='date', y='4', color='4', title='Precipitation [mm/day]')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.bar(dfs['CHIRPS']['precipitation_ma'], x='date', y='4', color='4', title='Precipitation MA [mm/day]')
fig.show()

##MODIS (LST)

###2022_02_20

In [ ]:
indices = [ix for ix in dfs['MODIS'].keys() if 'ma' not in ix and 'err' not in ix and ix != 'quality']
indices

In [ ]:
for ix in indices:
  dfs['MODIS'][ix + '_ma'] = dfs['MODIS'][ix].copy()

In [ ]:
dfs['MODIS'].keys()

In [ ]:
len(_)

In [ ]:
for ix in indices:
  dfs['MODIS'][ix + '_ma'] = dfs['MODIS'][ix + '_ma'].groupby([pd.Grouper(key='date', freq='W-MON')]).mean().reset_index().sort_values('date')
  dfs['MODIS'][ix + '_ma'].sort_index(inplace=True)

In [ ]:
print(dfs['MODIS']['lst_c_ma'])

In [ ]:
print(dfs['MODIS']['lst_c'])

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['MODIS']['lst_c']['date'], y=dfs['MODIS']['lst_c']['10'], title='LST [C]')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['MODIS']['lst_c_ma']['date'], y=dfs['MODIS']['lst_c_ma']['10'], title='LST MA [C]')
fig.show()

###Average Weekly

In [ ]:
dfs['MODIS']['lst_ma'] = dfs['MODIS']['lst'].copy()

In [ ]:
dfs['MODIS'].keys()

In [ ]:
dfs['MODIS']['lst_ma'] = dfs['MODIS']['lst_ma'].groupby([pd.Grouper(key='date', freq='W-MON')]).mean().reset_index().sort_values('date')
dfs['MODIS']['lst_ma'].sort_index(inplace=True)

In [ ]:
print(dfs['MODIS']['lst_ma'])

In [ ]:
print(dfs['MODIS']['lst'])

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['MODIS']['lst']['date'], y=dfs['MODIS']['lst']['10'], title='LST [K]')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['MODIS']['lst_ma']['date'], y=dfs['MODIS']['lst_ma']['10'], title='LST MA [K]')
fig.show()

##Sentinel-1

###Moving Average of ~ 2 Readings / 13 days

In [ ]:
indices = [ix for ix in dfs['S1'].keys() if 'ma' not in ix and 'err' not in ix]
indices

In [ ]:
for ix in indices:
  dfs['S1'][ix + '_ma'] = dfs['S1'][ix].copy()

In [ ]:
dfs['S1'].keys()

In [ ]:
len(_)

In [ ]:
for ix in indices:
  dfs['S1'][ix + '_ma'].set_index('date', inplace=True)
  for col in dfs['S1'][ix + '_ma'].columns:
    dfs['S1'][ix + '_ma'][col] = dfs['S1'][ix + '_ma'][col].rolling('13D', min_periods=1).mean()
  dfs['S1'][ix + '_ma'].reset_index(inplace=True)
  dfs['S1'][ix + '_ma'].sort_index(inplace=True)

In [ ]:
print(dfs['S1']['b_asc_vh'])

In [ ]:
print(dfs['S1']['b_asc_vh'])

In [ ]:
print(dfs['S1']['b_asc_vh_ma'])

In [ ]:
print(dfs['S1']['b_des_vh_ma'])

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S1']['b_asc_vh']['date'], y=dfs['S1']['b_asc_vh']['2'], title='S1')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S1']['b_asc_vh_ma']['date'], y=dfs['S1']['b_asc_vh_ma']['2'], title='S1 MA')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S1']['b_des_vh']['date'], y=dfs['S1']['b_des_vh']['2'], title='S1')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S1']['b_des_vh_ma']['date'], y=dfs['S1']['b_des_vh_ma']['2'], title='S1 MA')
fig.show()

##Sentinel-2 (CI, NDVI, NDWI)

###Average of Cloud-free days between all points

In [ ]:
cloud_free_days = dfs['S2']['cloudfree'].drop(['date'], 1).astype(bool).sum(axis=0)
# percent = cloud_free_days / len(dfs['S2']['cloudfree']) * 100
print("Total days: ", len(dfs['S2']['cloudfree']))
print()
# print("[%] cloud-free days @ each point: ")
# print(percent)
print("# cloud-free days @ each point:")
print(f'{cloud_free_days}')
print()
print("Range of cloud-free days, Average:")
min_days = cloud_free_days.min()
max_days = cloud_free_days.max()
avg_days = cloud_free_days.mean()
print(f'[{min_days} - {max_days}], {avg_days} Days.')
print()
# print("Average [%] of cloud-free days between all points: ", percent.mean())
point_percent = cloud_free_days / len(dfs['S2']['cloudfree']) * 100
print("[%] of cloud-free days @ each point:")
print(f'{point_percent}')
print()
print("Range of [%] cloud-free days, Average:")
min_per = point_percent.min()
max_per = point_percent.max()
avg_per = point_percent.mean()
print(f'[{min_per} - {max_per}], {avg_per} [%].')
print()

###Moving Average of ~ 4 Readings / 16 Days

In [ ]:
indices = [ix for ix in dfs['S2'].keys() if ix != 'cloudfree' and 'ma' not in ix and 'err' not in ix]
indices

In [ ]:
for ix in indices:
  dfs['S2'][ix + '_ma'] = dfs['S2'][ix].copy()

In [ ]:
dfs['S2'].keys()

In [ ]:
for ix in indices:
  dfs['S2'][ix + '_ma'].set_index('date', inplace=True)
  for col in dfs['S2'][ix + '_ma'].columns:
    dfs['S2'][ix + '_ma'][col] = dfs['S2'][ix + '_ma'][col].rolling('16D', min_periods=1).mean()
  dfs['S2'][ix + '_ma'].reset_index(inplace=True)
  dfs['S2'][ix + '_ma'].sort_index(inplace=True)

In [ ]:
print(dfs['S2']['ndvi'])

In [ ]:
print(dfs['S2']['ndvi_ma'][250:296])

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S2']['ndvi']['date'], y=dfs['S2']['ndvi']['2'], title='NDVI')
fig.show()

In [ ]:
fig = go.Figure()

fig = px.line(x=dfs['S2']['ndvi_ma']['date'], y=dfs['S2']['ndvi_ma']['2'], title='NDVI MA')
fig.show()

#Charts' Examples

##CHIRPS (Precipitation)

In [ ]:
dfs['CHIRPS'].keys()

In [ ]:
fig = go.Figure()

for col in range(2, 3):
  for ix in dfs['CHIRPS'].keys():
    fig.add_trace(go.Scatter(x=dfs['CHIRPS'][ix]['date'], 
                             y=dfs['CHIRPS'][ix][str(col)], 
                             mode='lines+markers', 
                             name=ix + " @ point #" + str(col)))

fig.update_layout(title='Precipitation @ AOI',
                  xaxis_title='Date',
                  yaxis_title='Precipitation [mm]')
fig.show()

##MODIS (LST)

In [ ]:
dfs['MODIS'].keys()

In [ ]:
fig = go.Figure()

for col in range(2, 3):
  for ix in dfs['MODIS'].keys():
    fig.add_trace(go.Scatter(x=dfs['MODIS'][ix]['date'], 
                             y=dfs['MODIS'][ix][str(col)], 
                             mode='lines+markers', 
                            name=ix + " @ point #" + str(col)))

fig.update_layout(title='LST @ AOI',
                  xaxis_title='Date',
                  yaxis_title='LST')
fig.show()

##Sentinel-1

In [ ]:
dfs['S1'].keys()

In [ ]:
fig = go.Figure()
satellite = 'b'
orbit = 'asc'

for col in range(0, 2):
  for ix in dfs['S1'].keys():
    if ix[0] == satellite:
      if orbit in ix:
        fig.add_trace(go.Scatter(x=dfs['S1'][ix]['date'], 
                                  y=dfs['S1'][ix][str(col)], 
                                  mode='lines+markers', 
                                  name=ix + " @ point #" + str(col)))

fig.update_layout(title='Angle, VH, VV @ AOI (Satellite: ' + satellite + ', Orbit: ' + orbit +')',
                  xaxis_title='Date',
                  yaxis_title='Angle, VH, VV')
fig.show()

In [ ]:
fig = go.Figure()
satellite = 'b'
orbit = 'des'

for col in range(0, 2):
  for ix in dfs['S1'].keys():
    if ix[0] == satellite:
      if orbit in ix:
        fig.add_trace(go.Scatter(x=dfs['S1'][ix]['date'], 
                                  y=dfs['S1'][ix][str(col)], 
                                  mode='lines+markers', 
                                  name=ix + " @ point #" + str(col)))

fig.update_layout(title='Angle, VH, VV @ AOI (Satellite: ' + satellite + ', Orbit: ' + orbit +')',
                  xaxis_title='Date',
                  yaxis_title='Angle, VH, VV')
fig.show()

##Sentinel-2 (CI, NDVI, NDWI)

In [ ]:
dfs['S2'].keys()

In [ ]:
fig = go.Figure()

for col in range(2, 3):
  for ix in dfs['S2'].keys():
    if 'cloudfree' not in ix:
      fig.add_trace(go.Scatter(x=dfs['S2'][ix]['date'], 
                               y=dfs['S2'][ix][str(col)], 
                               mode='lines+markers', 
                               name=ix + " @ point #" + str(col)))

fig.update_layout(title='CI, NDVI, NDWI @ AOI',
                  xaxis_title='Date',
                  yaxis_title='CI, NDVI, NDWI')
fig.show()

#Writing MA Time Series Dataframes to Disc

In [ ]:
for s in sources:
  indices = [ix for ix in dfs[s].keys() if 'ma' in ix]
  print(s)
  print(indices)
  print()
  for ix in indices:
    dfs[s][ix].to_csv(dir + s + '/' + ix[:-3] + '__ma.csv')

#Normalize MA Time Series Dataframes + Writing to Disc

In [ ]:
for s in sources:
  indices = [ix for ix in dfs[s].keys() if 'ma' in ix and 'norm' not in ix]
  print(s)
  print(indices)
  print()
  for ix in indices:
    dfs[s][ix + '_norm'] = dfs[s][ix].copy()
    df_values = dfs[s][ix + '_norm'].iloc[:, 1:].copy()
    dfs[s][ix + '_norm'].iloc[:, 1:] = (df_values - np.nanmean(df_values.values)) / np.nanstd(df_values.values)
    dfs[s][ix + '_norm'].sort_index(inplace=True)
    dfs[s][ix + '_norm'].to_csv(dir + s + '/' + ix[:-3] + '__ma__norm.csv')
    # print(ix + '_norm')
    # # print(dfs[s][ix + '_norm'])
    # print(np.nanmean(dfs[s][ix + '_norm'].values))
    # print(np.nanstd(dfs[s][ix + '_norm'].values))
    # print()

In [ ]:
for s in sources:
  indices = [ix for ix in dfs[s].keys()]
  print(s)
  print(indices)
  print()

In [ ]:
dfs['S1']['b_asc_vh']

In [ ]:
dfs['S1']['b_asc_vh_ma']

In [ ]:
dfs['S1']['b_asc_vh_ma_norm']

In [ ]:
dfs['S2']['ndvi']
# dfs['MODIS']['lst_c']

In [ ]:
dfs['S2']['cloudfree']

In [ ]:
dfs['S2']['ndvi_ma']
# dfs['MODIS']['lst_c_ma']

In [ ]:
dfs['S2']['ndvi_ma_norm']
# dfs['MODIS']['lst_c_ma_norm']

In [ ]:
df = dfs['S1']['b_asc_vh_ma_norm']
# df = dfs['MODIS']['lst_c_ma_norm']

print(np.nanmean(df.iloc[:, 1:].values))
print(np.nanstd(df.iloc[:, 1:].values))

In [ ]:
df = dfs['S2']['ndwi_ma_norm']
# df = dfs['MODIS']['lst_c_ma_norm']

print(np.nanmean(df.iloc[:, 1:].values))
print(np.nanstd(df.iloc[:, 1:].values))

In [ ]:
df = df.dropna()

print(df.shape[0])
print(df.shape[1])
print(np.nanmean(df.iloc[:, 1:].values))
print(np.nanstd(df.iloc[:, 1:].values))